In [2]:
import pandas as pd
import numpy as np
import tensorflow as tf
from collections import Counter
from tensorflow.keras import Sequential
import importlib.util
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Conv2D,MaxPool2D,Softmax,Flatten
from tensorflow.keras.datasets import mnist
from tensorflow.keras.preprocessing.image import ImageDataGenerator

from tensorflow.keras import layers, models, callbacks, utils,optimizers


In [7]:
def to_float32(img):
    return img.astype(np.float32)
trainD=ImageDataGenerator(
rescale=1./255,rotation_range=10,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    zoom_range=0.1,
    horizontal_flip=False,
    preprocessing_function=to_float32,
    fill_mode='nearest')        # Fill edges with reflection (better than black))
df_train = trainD.flow_from_directory(
    r"C:\Users\DAR-MG\Desktop\Hammad\mnist_dataset\train",
    target_size=(28, 28),
    color_mode="grayscale",      # MNIST is single-channel
    batch_size=32,
    class_mode="sparse"          # <-- not the loss name
)

Found 40000 images belonging to 10 classes.


In [8]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

def to_float32(img):
    return img.astype('float32')

testD = ImageDataGenerator(
    rescale=1.0/255.0,
    preprocessing_function=to_float32
)

df_test = testD.flow_from_directory(
    r"C:\Users\DAR-MG\Desktop\Hammad\mnist_dataset\test",
    target_size=(28, 28),
    color_mode="grayscale",
    batch_size=32,
    class_mode="sparse",
    shuffle=False  # typically you don't shuffle test data
)

valD = ImageDataGenerator(
    rescale=1.0/255.0,
    preprocessing_function=to_float32
)

df_val = valD.flow_from_directory(
    r"C:\Users\DAR-MG\Desktop\Hammad\mnist_dataset\val",
    target_size=(28, 28),
    color_mode="grayscale",
    batch_size=32,
    class_mode="sparse",
    shuffle=False
)


Found 10000 images belonging to 10 classes.
Found 10000 images belonging to 10 classes.


In [9]:

def create_model():
    model = models.Sequential()
    model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)))
    model.add(layers.Conv2D(32, (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))

    model.add(layers.Conv2D(64, (3, 3), activation='relu'))
    model.add(layers.Conv2D(64, (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))

    model.add(layers.Flatten())
    model.add(layers.Dense(512, activation='relu'))
    model.add(layers.Dense(128, activation='relu'))
    model.add(layers.Dense(256, activation='relu'))
    model.add(layers.Dense(32, activation='relu'))
    model.add(layers.Dropout(0.1))
    model.add(layers.Dense(10, activation='softmax'))

    return model

create_model().summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 26, 26, 32)        320       
                                                                 
 conv2d_1 (Conv2D)           (None, 24, 24, 32)        9248      
                                                                 
 max_pooling2d (MaxPooling2D  (None, 12, 12, 32)       0         
 )                                                               
                                                                 
 conv2d_2 (Conv2D)           (None, 10, 10, 64)        18496     
                                                                 
 conv2d_3 (Conv2D)           (None, 8, 8, 64)          36928     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 4, 4, 64)         0         
 2D)                                                    

In [ ]:
from tensorflow_addons.optimizers import AdaBelief
import pickle  
import tensorflow as tf
from tensorflow_addons.optimizers import AdaBelief
from tensorflow.keras.callbacks import EarlyStopping
import pickle

# 1. Learning rates
learning_rates = {
    'Adam_AMSGrad': 5e-4,
    'AdaBelief': 1e-3,
    'Adam': 1e-4,
    'SGD': 1e-2,
    'RMSprop': 1e-3,
    'Adagrad': 1e-2
}

hnadam_lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=1e-3,
    decay_steps=1000,
    decay_rate=0.9
)

optimizers_dict = {
    'Adam_AMSGrad': lambda: tf.keras.optimizers.Adam(learning_rate=learning_rates['Adam_AMSGrad'], amsgrad=True),
    'AdaBelief': lambda: AdaBelief(learning_rate=learning_rates['AdaBelief']),
    'Adam': lambda: tf.keras.optimizers.Adam(learning_rate=learning_rates['Adam']),
    'SGD': lambda: tf.keras.optimizers.SGD(learning_rate=learning_rates['SGD'], momentum=0.9),
    'RMSprop': lambda: tf.keras.optimizers.RMSprop(learning_rate=learning_rates['RMSprop']),
    'Adagrad': lambda: tf.keras.optimizers.Adagrad(learning_rate=learning_rates['Adagrad']),
}


histories = {}

for name, opt_fn in optimizers_dict.items():
    print(f"\nTraining with {name}…")
    
    model = create_model()
    model.compile(
        optimizer=opt_fn(),
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )
    
    hist = model.fit(
        df_train,
        epochs=100,
        batch_size=32,
        validation_data=df_val,
        verbose=1
    )
    
    # Save history in dictionary and also to file
    histories[name] = hist.history
    
    with open(f'history_{name}.pkl', 'wb') as f:
        pickle.dump(hist.history, f)

    # Save model
    model.save(f"model_{name}.h5")



Training with HN_Adam…
Epoch 1/100
1250/1250 [==============================] - 72s 57ms/step - loss: 0.3189 - accuracy: 0.9008 - val_loss: 0.0631 - val_accuracy: 0.9840
Epoch 2/100
1250/1250 [==============================] - 64s 51ms/step - loss: 0.0959 - accuracy: 0.9732 - val_loss: 0.0523 - val_accuracy: 0.9866
Epoch 3/100
1250/1250 [==============================] - 57s 46ms/step - loss: 0.0664 - accuracy: 0.9816 - val_loss: 0.0377 - val_accuracy: 0.9891
Epoch 4/100
1250/1250 [==============================] - 57s 45ms/step - loss: 0.0499 - accuracy: 0.9855 - val_loss: 0.0395 - val_accuracy: 0.9897
Epoch 5/100
1250/1250 [==============================] - 57s 46ms/step - loss: 0.0418 - accuracy: 0.9876 - val_loss: 0.0340 - val_accuracy: 0.9914
Epoch 6/100
1250/1250 [==============================] - 57s 46ms/step - loss: 0.0354 - accuracy: 0.9900 - val_loss: 0.0287 - val_accuracy: 0.9919
Epoch 7/100
1250/1250 [==============================] - 58s 46ms/step - loss: 0.0306 - accura

In [10]:
from tensorflow_addons.optimizers import AdaBelief
import pickle  
import tensorflow as tf
from tensorflow_addons.optimizers import AdaBelief
from tensorflow.keras.callbacks import EarlyStopping
import pickle

# 1. Learning rates
learning_rates = {
    'Adagrad': 1e-2
}

hnadam_lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=1e-3,
    decay_steps=1000,
    decay_rate=0.9
)

optimizers_dict = {

    'Adagrad': lambda: tf.keras.optimizers.Adagrad(learning_rate=learning_rates['Adagrad']),
}


histories = {}

for name, opt_fn in optimizers_dict.items():
    print(f"\nTraining with {name}…")
    
    model = create_model()
    model.compile(
        optimizer=opt_fn(),
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )
    
    hist = model.fit(
        df_train,
        epochs=100,
        batch_size=32,
        validation_data=df_val,
        verbose=1
    )
    
    # Save history in dictionary and also to file
    histories[name] = hist.history
    
    with open(f'history_{name}.pkl', 'wb') as f:
        pickle.dump(hist.history, f)

    # Save model
    model.save(f"model_{name}.h5")



Training with Adagrad…
Epoch 1/100
1250/1250 [==============================] - 69s 54ms/step - loss: 0.8726 - accuracy: 0.7072 - val_loss: 0.1572 - val_accuracy: 0.9523
Epoch 2/100
1250/1250 [==============================] - 61s 49ms/step - loss: 0.1984 - accuracy: 0.9407 - val_loss: 0.0945 - val_accuracy: 0.9709
Epoch 3/100
1250/1250 [==============================] - 55s 44ms/step - loss: 0.1354 - accuracy: 0.9600 - val_loss: 0.0637 - val_accuracy: 0.9808
Epoch 4/100
1250/1250 [==============================] - 55s 44ms/step - loss: 0.1098 - accuracy: 0.9672 - val_loss: 0.0660 - val_accuracy: 0.9801
Epoch 5/100
1250/1250 [==============================] - 55s 44ms/step - loss: 0.0945 - accuracy: 0.9721 - val_loss: 0.0552 - val_accuracy: 0.9843
Epoch 6/100
1250/1250 [==============================] - 55s 44ms/step - loss: 0.0838 - accuracy: 0.9760 - val_loss: 0.0653 - val_accuracy: 0.9807
Epoch 7/100
1250/1250 [==============================] - 55s 44ms/step - loss: 0.0758 - accura

In [12]:
from tensorflow.keras.models import load_model

# Paths for your saved models
model_paths = {
    'HN_Adam': 'model_HN_Adam.h5',
    'AdaBelief': 'model_AdaBelief.h5',
    'Adam': 'model_Adam.h5',
    'Adam_AMSGrad': 'model_Adam_AMSGrad.h5',
    'SGD': 'model_SGD.h5',
    'RMSprop': 'model_RMSprop.h5',
    'Adagrad': 'model_Adagrad.h5',
}

# Dictionary to store test accuracies
test_accuracies = {}

# Load and evaluate each model
for name, model_path in model_paths.items():
    print(f"\nLoading and evaluating {name} model...")

    if name == 'HN_Adam':
        model = load_model(model_path, custom_objects={'HN_Adam': HN_Adam})
    else:
        model = load_model(model_path)

    # Evaluate on the test data
    loss, acc = model.evaluate(df_test, verbose=0)
    test_accuracies[name] = acc
    print(f"✅ {name} Test Accuracy: {acc:.4f}")



Loading and evaluating HN_Adam model...
✅ HN_Adam Test Accuracy: 0.9958

Loading and evaluating AdaBelief model...
✅ AdaBelief Test Accuracy: 0.9930

Loading and evaluating Adam model...
✅ Adam Test Accuracy: 0.9953

Loading and evaluating Adam_AMSGrad model...
✅ Adam_AMSGrad Test Accuracy: 0.9957

Loading and evaluating SGD model...
✅ SGD Test Accuracy: 0.9941

Loading and evaluating RMSprop model...
✅ RMSprop Test Accuracy: 0.1135

Loading and evaluating Adagrad model...
✅ Adagrad Test Accuracy: 0.9946
